# Keras Tuner Bars

In [1]:
import h5py
import numpy as np
import tensorflow
from tensorflow import keras
import kerastuner

print("tensorflow", tensorflow.__version__)
print("keras", keras.__version__)
print("keras-tuner", kerastuner.__version__)

tensorflow 2.1.0
keras 2.2.4-tf
keras-tuner 1.0.1


In [2]:
class HDF5DataGeneratorBars(keras.utils.Sequence):
    def __init__(self, file, num_classes, num_dp, batch_size=200, num_batches=None):
        self.file = h5py.File(file, "r")
        self.batch_size = batch_size
        self.num_classes = num_classes
        self.num_dp = num_dp

        if num_batches == None:
            self.num_batches = int(
                np.floor(self.file["multiplicity"].len() / self.batch_size)
            )
        else:
            self.num_batches = num_batches

    def __len__(self):
        return self.num_batches

    def __getitem__(self, index):
        x, y = self.getbatch(index)
        np.place(y, y >= self.num_classes, self.num_classes - 1)  # FIXME
        y = keras.utils.to_categorical(y, num_classes=self.num_classes)
        return x, y

    def getbatch(self, index):
        "Generate one batch of data"
        start = self.batch_size * index
        end = self.batch_size * (index + 1)
        x = np.array(self.file["flatfeatures"][start:end])
        # multiplicity column 2 -> label = 'nPH'
        y = np.array(self.file["multiplicity"][start:end, [2]].ravel())
        return x, y

In [3]:
class Model(kerastuner.HyperModel):
    def __init__(self, num_classes):
        self.num_classes = num_classes

    def build(self, hp):
        model = keras.models.Sequential()
        for i in range(hp.Int("num_layers", 1, 3)):
            model.add(
                keras.layers.Dense(
                    units=hp.Int(
                        f"units_{i}", min_value=1000, max_value=10000, step=3000
                    ),
                    activation="relu",
                )
            )
        model.add(
            keras.layers.Dense(
                units=hp.Int(f"units_last", min_value=50, max_value=550, step=100),
                activation="relu",
            )
        )
        model.add(keras.layers.Dense(units=self.num_classes, activation="softmax"))

        #optimizer = keras.optimizers.Adam()
        optimizer = hp.Choice('optimizer', ['adam', 'adagrad', 'adadelta', 'SGD'])
        loss = keras.losses.CategoricalCrossentropy()

        model.compile(loss=loss, optimizer=optimizer, metrics=["accuracy"])

        return model

In [4]:
num_dp = 30
max_neutrons = 5

file = f"data/600AMeV_{num_dp}dp.bars-shuffled.h5"
generator = HDF5DataGeneratorBars(file, num_classes=max_neutrons + 1, num_dp=num_dp)
model = Model(max_neutrons + 1)

# tuner = kerastuner.tuners.RandomSearch(
#     model,
#     objective="accuracy",
#     max_trials=20,
#     executions_per_trial=1,
#     directory="tmp-kerastuner",
#     project_name="tunebars5",
# )

tuner = kerastuner.tuners.Hyperband(
    model,
    objective="accuracy",
    max_epochs=5,
    executions_per_trial=1,
    directory="tmp-kerastuner",
    project_name="tunebars7",
)
tuner.search_space_summary()

INFO:tensorflow:Reloading Oracle from existing project tmp-kerastuner\tunebars6\oracle.json
INFO:tensorflow:Reloading Tuner from tmp-kerastuner\tunebars6\tuner0.json


In [5]:
tuner.search(
    generator,
    verbose=2,
)

Train for 25000 steps
Epoch 1/3
25000/25000 - 4811s - loss: nan - accuracy: 0.0051
Epoch 2/3
25000/25000 - 3575s - loss: nan - accuracy: 0.0048
Epoch 3/3
25000/25000 - 1798s - loss: nan - accuracy: 0.0048


c:\users\jan\.virtualenvs\futurereco2-s-fscxbi\lib\site-packages\kerastuner\engine\metrics_tracking.py:92: RuntimeWarning: All-NaN axis encountered
  return np.nanmin(values)


Train for 25000 steps
Epoch 1/3
25000/25000 - 2316s - loss: 311329.6748 - accuracy: 0.2815
Epoch 2/3
25000/25000 - 2917s - loss: 162271.2980 - accuracy: 0.3635
Epoch 3/3
25000/25000 - 2806s - loss: 120142.0960 - accuracy: 0.4116


Train for 25000 steps
Epoch 1/3
25000/25000 - 1575s - loss: 220.2343 - accuracy: 0.3088
Epoch 2/3
25000/25000 - 1220s - loss: 1.3100 - accuracy: 0.4593
Epoch 3/3
25000/25000 - 1197s - loss: 2.6012 - accuracy: 0.2776


Train for 25000 steps
Epoch 1/3
25000/25000 - 1426s - loss: nan - accuracy: 0.0062
Epoch 2/3
25000/25000 - 1531s - loss: nan - accuracy: 0.0048
Epoch 3/3
25000/25000 - 1678s - loss: nan - accuracy: 0.0048


...

Train for 25000 steps
Epoch 8/20
25000/25000 - 2931s - loss: 61.4342 - accuracy: 0.5879
Epoch 9/20
25000/25000 - 2919s - loss: 0.7327 - accuracy: 0.6788
Epoch 10/20
25000/25000 - 2909s - loss: 0.6579 - accuracy: 0.7104
Epoch 11/20
25000/25000 - 2916s - loss: 0.5962 - accuracy: 0.7402
Epoch 12/20
25000/25000 - 2920s - loss: 0.5347 - accuracy: 0.7707
Epoch 13/20
25000/25000 - 2948s - loss: 0.4698 - accuracy: 0.8025
Epoch 14/20
25000/25000 - 2982s - loss: 0.4045 - accuracy: 0.8334
Epoch 15/20
25000/25000 - 2939s - loss: 0.3423 - accuracy: 0.8612
Epoch 16/20
25000/25000 - 3025s - loss: 0.2874 - accuracy: 0.8848
Epoch 17/20
25000/25000 - 2978s - loss: 0.2446 - accuracy: 0.9018
Epoch 18/20
25000/25000 - 2977s - loss: 0.2123 - accuracy: 0.9138
Epoch 19/20
25000/25000 - 2982s - loss: 0.1886 - accuracy: 0.9223
Epoch 20/20
25000/25000 - 2971s - loss: 0.1714 - accuracy: 0.9280


Train for 25000 steps
Epoch 8/20
25000/25000 - 2586s - loss: 385195.1603 - accuracy: 0.6131
Epoch 9/20
25000/25000 - 2701s - loss: 13.0728 - accuracy: 0.6543
Epoch 10/20
25000/25000 - 2645s - loss: 46131.7522 - accuracy: 0.6826
Epoch 11/20
25000/25000 - 2628s - loss: 11718.0588 - accuracy: 0.7033
Epoch 12/20
25000/25000 - 2579s - loss: 304950.7912 - accuracy: 0.7220
Epoch 13/20
25000/25000 - 2567s - loss: 111371.6720 - accuracy: 0.7389
Epoch 14/20
25000/25000 - 2519s - loss: 10413.8709 - accuracy: 0.7557
Epoch 15/20
25000/25000 - 2547s - loss: 233082.1186 - accuracy: 0.7708
Epoch 16/20
25000/25000 - 2543s - loss: 477208.1796 - accuracy: 0.7829
Epoch 17/20
25000/25000 - 2514s - loss: 623634.0330 - accuracy: 0.7863
Epoch 18/20
25000/25000 - 2511s - loss: 26720.5920 - accuracy: 0.8031
Epoch 19/20
25000/25000 - 2513s - loss: 161490.5343 - accuracy: 0.8114
Epoch 20/20
25000/25000 - 2511s - loss: 22246.2147 - accuracy: 0.8264


Train for 25000 steps
Epoch 1/20
25000/25000 - 694s - loss: 209012.3678 - accuracy: 0.2524
Epoch 2/20
25000/25000 - 627s - loss: 110903.8108 - accuracy: 0.3334
Epoch 3/20
25000/25000 - 635s - loss: 76309.7761 - accuracy: 0.3850
Epoch 4/20
25000/25000 - 630s - loss: 53264.9751 - accuracy: 0.4234
Epoch 5/20
25000/25000 - 628s - loss: 41359.3142 - accuracy: 0.4516
Epoch 6/20
25000/25000 - 628s - loss: 31977.7959 - accuracy: 0.4744
Epoch 7/20
25000/25000 - 628s - loss: 26532.5076 - accuracy: 0.4960
Epoch 8/20
25000/25000 - 628s - loss: 22538.7669 - accuracy: 0.5172
Epoch 9/20
25000/25000 - 627s - loss: 18812.8355 - accuracy: 0.5388
Epoch 10/20
25000/25000 - 626s - loss: 16492.7266 - accuracy: 0.5603
Epoch 11/20
25000/25000 - 626s - loss: 14218.6574 - accuracy: 0.5803
Epoch 12/20
25000/25000 - 627s - loss: 11159.1173 - accuracy: 0.5962
Epoch 13/20
25000/25000 - 629s - loss: 3026.7954 - accuracy: 0.6053
Epoch 14/20
25000/25000 - 626s - loss: 651.0188 - accuracy: 0.6173
Epoch 15/20
25000/2500

Train for 25000 steps
Epoch 1/20
25000/25000 - 1097s - loss: 1.6613 - accuracy: 0.2040
Epoch 2/20
25000/25000 - 1100s - loss: 1.6311 - accuracy: 0.2041
Epoch 3/20
25000/25000 - 1103s - loss: 1.6311 - accuracy: 0.2042
Epoch 4/20
25000/25000 - 1054s - loss: 1.6311 - accuracy: 0.2040
Epoch 5/20
25000/25000 - 1052s - loss: 1.6311 - accuracy: 0.2039
Epoch 6/20
25000/25000 - 1053s - loss: 1.6311 - accuracy: 0.2040
Epoch 7/20
25000/25000 - 1096s - loss: 1.6311 - accuracy: 0.2044
Epoch 8/20
25000/25000 - 1059s - loss: 1.6311 - accuracy: 0.2040
Epoch 9/20
25000/25000 - 1052s - loss: 1.6311 - accuracy: 0.2041
Epoch 10/20
25000/25000 - 1052s - loss: 1.6311 - accuracy: 0.2040
Epoch 11/20
25000/25000 - 1053s - loss: 1.6311 - accuracy: 0.2040
Epoch 12/20
25000/25000 - 1172s - loss: 1.6311 - accuracy: 0.2040
Epoch 13/20


KeyboardInterrupt: 

In [6]:
tuner.results_summary()